## Json2Mask
Extract a mask file from a Json

Input: Mask Json files

Output: Mask png files

In [104]:
from PIL import Image, ImageDraw
import os
import json

home = '../preliminary'
paths = [os.path.join(home, name) for name in os.listdir(home)]
for path in paths:
    with open(path) as f:
        data = json.load(f)

    # Create a background image.
    img = Image.new('RGB', [data['imageWidth'], data['imageHeight']], '#000000')
    _img = ImageDraw.Draw(img)

    # Draw polygons.
    for shape in data['shapes']:
        _t = [tuple(x) for x in shape['points']] # Point type must be tuple.
        # TODO: Separate color by a label. (Currently only oil exists.)
        _img.polygon(_t, fill="#ffffff")  # Polygon color is white.

    # Save a polygon image.
    img.save(os.path.join(home, data['imagePath']))

## Checking number of object patches
Check how many patches actually include objects.

Input: Mask png files

In [198]:
def get_subimg(img, patch_size):
    """
    Get a sub-image which of the minimum size to include patch areas
    to count number of object patch images faster.
    Size of the Sub-image is a mutiple of patch_size.

    Args:
        img(ndarray): Original boolean image array. shape:(h, w)
        patch_size(int): Patch size.
    Returns:
        subimg(ndarray): Sub-image. shape: (h, w)
    """
    y, x= np.where(img)
    tx = min(x)
    ty = min(y)
    bx = max(x)
    by = max(y)

    sub_width = bx - tx + 1
    if sub_width % patch_size != 0:
        sub_width = patch_size * (sub_width // patch_size + 1)
            
    sub_height = by - ty + 1
    if sub_height % patch_size != 0:
        sub_height = patch_size * (sub_height // patch_size + 1)
        
    (height, width) = img.shape
    cx = int((bx - tx) / 2) + tx
    cy = int((by - ty) / 2) + ty

    tx = cx - int(sub_width / 2)
    bx = tx + sub_width - 1
    ty = cy - int(sub_height / 2)
    by = ty + sub_height - 1

    if tx < 0:
        tx = 0
        bx = sub_width
    elif bx >= width:
        tx = width - sub_width
        bx = width - 1

    if ty < 0:
        ty = 0
        by = sub_height
    elif by >= height:
        ty = height - sub_height
        by = height - 1
    
    subimg = img[ty:by+1, tx:bx+1]
    return subimg

In [209]:
import cv2
import os

home = '../preliminary'
paths = [os.path.join(home, name) for name in os.listdir(home) if 'png' in name]
patch_size = 224
ovr_ratio = 0.5 # ovr_ratio must be in range of [0, 1)

num_patches = 0
for path in paths:
    img = cv2.imread(path)  # black(0,0,0) or white(255,255,255) image
    img = np.array(img[...,0], dtype=bool)
    img = get_subimg(img, patch_size)
    
    height, width = img.shape
    for c in range(0, height, patch_size - int(patch_size * ovr_ratio)):
        for r in range(0, width, patch_size - int(patch_size * ovr_ratio)):
            if True in img[c:c+patch_size, r:r+patch_size]:
                num_patches += 1

print('Num. object patches: ', num_patches)

Num. object patches:  1100
